# 1. IMPORTS

## 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## 1.2 Data

In [2]:
folder = "../temporary/"

df_events=pd.read_csv(folder + "usa_historical_events.csv")
df_treated=pd.read_csv(folder + "cleaned_data.csv",index_col = 'Wikipedia movie ID')

PLOT_SUMMARIES = "../data/plot_summaries.txt"

with open(PLOT_SUMMARIES, 'r', encoding='utf-8') as file:
    plot_summaries = file.readlines()

# 2. PLOT TONE SCORE

In [3]:
# For this, we use the "VADER sentiment analysis" library which contains a dictionary. Each of the words inside are associated to a polarity score depending on how positive or negative they are. One total 
#score is computed per summary, and then it is normalized and takes a value between -1 and +1 to categorize the text as positive, negative or neutral. 

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    score = analyzer.polarity_scores(text)
    compound_score = score['compound']
    if compound_score >= 0.1:
        sentiment = 'positive'
    elif compound_score <= -0.1:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    return sentiment, compound_score

In [4]:
data = []

for line in plot_summaries:
    parts = line.strip().split('\t', 1) # Split each line by the tab character
    if len(parts) == 2:
        movie_id, summary = parts
        sentiment, compound_score = get_sentiment(summary)
        data.append([movie_id, summary, sentiment, compound_score])

df_sent = pd.DataFrame(data, columns=['ID', 'Summary', 'Sentiment', 'Compound Score'])
df_sent = df_sent.set_index('ID')

df_sent.head(10)

,Summary,Sentiment,Compound Score
ID,,,
23890098,"Shlykov, a hard-working taxi driver and Lyosha...",neutral,0.0083
31186339,The nation of Panem consists of a wealthy Capi...,negative,-0.9941
20663735,Poovalli Induchoodan is sentenced for six yea...,negative,-0.9867
2231378,"The Lemon Drop Kid , a New York City swindler,...",negative,-0.6127
595909,Seventh-day Adventist Church pastor Michael Ch...,negative,-0.9538
5272176,The president is on his way to give a speech. ...,negative,-0.9946
1952976,"{{plot}} The film opens in 1974, as a young gi...",negative,-0.7904
24225279,"The story begins with Hannah, a young Jewish t...",negative,-0.5064
2462689,Infuriated at being told to write one final co...,negative,-0.9898


In [5]:
df_sent.index = df_sent.index.astype('int64')
df_merged = pd.merge(df_treated, df_sent, left_index=True, right_index=True, how='left')

In [8]:
df_events[['Sentiment', 'Compound Score']] = df_events.apply(
    lambda row: pd.Series(get_sentiment(row['Description'])) if pd.notna(row['Description']) else pd.Series(get_sentiment(row['Title'])),
    axis=1)

In [9]:
df_merged.head(10)

,Movie name,Movie release month,Movie release year,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Summary,Sentiment,Compound Score
Wikipedia movie ID,,,,,,,,,,,,
975900,Ghosts of Mars,8.0,2001.0,2001-08-24,2.477118e+07,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...","Set in the second half of the 22nd century, th...",negative,-0.9913
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2.0,2000.0,2000-02-16,NaN,95.0,['English'],['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",NaN,NaN,NaN
28463795,Brun bitter,NaN,1988.0,NaN,NaN,83.0,['Norwegian'],['Norway'],"['Crime Fiction', 'Drama']",NaN,NaN,NaN
9363483,White Of The Eye,NaN,1987.0,NaN,NaN,110.0,['English'],['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",A series of murders of rich young women throug...,negative,-0.9983
261236,A Woman in Flames,NaN,1983.0,NaN,NaN,106.0,['German'],['Germany'],['Drama'],"Eva, an upper class housewife, becomes frustra...",positive,0.9604
13696889,The Gangsters,5.0,1913.0,1913-05-29,NaN,35.0,"['Silent film', 'English']",['United States of America'],"['Short Film', 'Silent film', 'Indie', 'Black-...",NaN,NaN,NaN
18998739,The Sorcerer's Apprentice,NaN,2002.0,NaN,NaN,86.0,['English'],['South Africa'],"['Family Film', 'Fantasy', 'Adventure', 'World...","Every hundred years, the evil Morgana returns...",negative,-0.8885
10408933,Alexander's Ragtime Band,8.0,1938.0,1938-08-16,7.619573e+07,106.0,['English'],['United States of America'],"['Musical', 'Comedy', 'Black-and-white']",NaN,NaN,NaN
9997961,Contigo y aquí,NaN,1974.0,NaN,NaN,NaN,['Spanish'],['Argentina'],"['Musical', 'Drama', 'Comedy']",NaN,NaN,NaN


In [10]:
df_events.head(10)

,Date,Title,Description,Sentiment,Compound Score
0,1900-03-14,Gold Standard Act passed,Established gold as the only standard for rede...,positive,0.8934
1,1900-11-06,McKinley reelected president,NaN,neutral,0.0000
2,1901-02-25,Congress passes Platt Amendment,"The Platt Amendment, an amendment to a U.S. ar...",positive,0.3818
3,1901-02-25,J.P. Morgan creates United States Steel Corp,"1 of the most powerful bankers of his era, he ...",negative,-0.1796
4,1901-07-29,American Socialist Party formed,a multi-tendency democratic-socialist politica...,positive,0.9231
5,1901-09-06,McKinley assassinated; TR becomes president,Pres McKinley is shot at the Pan American Expo...,negative,-0.9382
6,1902-10-03,TR intervenes in anthracite coal strike,The strike of 1900 was the prelude to a larger...,negative,-0.1010
7,1903-01-01,Souls of Black Folk,"W. E. B. Du Bois’s landmark book of essays, So...",positive,0.4767
8,1903-11-19,Women's Trade Union League Formed,a U.S. org of both working class & more well-o...,positive,0.7430
9,1903-12-17,Wright Bros. make 1st successful flight,Orville Wright piloted the first powered airpl...,neutral,0.0000


# 3. EXPORT CLEAN DATASETS

In [12]:
df_merged.to_csv('../generated/cleaned_data.csv', index=True, encoding='utf-8')

In [13]:
df_events.to_csv('../generated/usa_historical_events.csv',index = False)